In [50]:
# 공통
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import norm
from sklearn.metrics import mean_squared_error
from dateutil.relativedelta import relativedelta
import datetime
import matplotlib.pyplot as plt

In [51]:
# Train 데이터 불러오기 (00년 ~ 21년)
# 2021년 Train / 2022년 2022Train

data = pd.read_csv('전체.csv', encoding='cp949')
# data = data[38:]

# data = data.drop(['price'], axis = 1)
# data = data.drop(['전체_temp'], axis = 1)
# data = data.drop(['전체_hum'], axis = 1)

data = data.drop(['강원_temp'], axis = 1)
data = data.drop(['경기_temp'], axis = 1)
data = data.drop(['경남_temp'], axis = 1)
data = data.drop(['경북_temp'], axis = 1)
data = data.drop(['광주_temp'], axis = 1)
data = data.drop(['대구_temp'], axis = 1)
data = data.drop(['대전_temp'], axis = 1)
data = data.drop(['부산_temp'], axis = 1)
data = data.drop(['서울_temp'], axis = 1)
data = data.drop(['세종_temp'], axis = 1)
data = data.drop(['울산_temp'], axis = 1)
data = data.drop(['충남_temp'], axis = 1)
data = data.drop(['충북_temp'], axis = 1)
data = data.drop(['전남_temp'], axis = 1)
data = data.drop(['전북_temp'], axis = 1)
data = data.drop(['제주_temp'], axis = 1)
data = data.drop(['인천_temp'], axis = 1)

data = data.drop(['강원_hum'], axis = 1)
data = data.drop(['경기_hum'], axis = 1)
data = data.drop(['경남_hum'], axis = 1)
data = data.drop(['경북_hum'], axis = 1)
data = data.drop(['광주_hum'], axis = 1)
data = data.drop(['대구_hum'], axis = 1)
data = data.drop(['대전_hum'], axis = 1)
data = data.drop(['부산_hum'], axis = 1)
data = data.drop(['서울_hum'], axis = 1)
data = data.drop(['세종_hum'], axis = 1)
data = data.drop(['울산_hum'], axis = 1)
data = data.drop(['충남_hum'], axis = 1)
data = data.drop(['충북_hum'], axis = 1)
data = data.drop(['전남_hum'], axis = 1)
data = data.drop(['전북_hum'], axis = 1)
data = data.drop(['제주_hum'], axis = 1)
data = data.drop(['인천_hum'], axis = 1)
    
data = data.set_index('date')
data = data.fillna(method='ffill')
data.tail(10)

,price,전체_temp,전체_hum
date,,,
2022-12-22,43915262.11,-0.069581,2.443615
2022-12-23,56974953.16,-0.068825,2.424751
2022-12-24,63610851.67,-0.068078,2.406034
2022-12-25,63931802.16,-0.067338,2.387328
2022-12-26,48637710.03,-0.066607,2.368320
2022-12-27,43408874.76,-0.065883,2.349998
2022-12-28,45710517.78,-0.065167,2.331975
2022-12-29,43882147.40,-0.064459,2.314035
2022-12-30,56931990.67,-0.063759,2.296192


In [52]:
# # 2021년 예측

# train = data[:'2021-11-30']['price']

# exog1 = data.drop('price', axis=1)[:'2021-11-30']
# exog1 = sm.add_constant(exog1)

# exog2 = data.drop('price', axis=1)['2021-12-01':]
# exog2 = sm.add_constant(exog2)

In [53]:
# 2022년 예측

train = data[:'2021-12-31']['price']

exog1 = data.drop('price', axis=1)[:'2021-12-31']

exog2 = data.drop('price', axis=1)['2022-01-01':]

In [54]:
# start = datetime.datetime.strptime("2021-01-01", "%Y-%m-%d")

# # 기준 날짜에 대한 연산이 필요할 때 dateutil.relativedelta모듈의 relativedelta함수 이용
# df_list = [start + relativedelta(days = x) for x in range(0, len(data)) ]

# # 그리드 서치(모델에게 가장 적합한 하이퍼 파라미터를 찾기) p, q

# import itertools

# # Define the p, d and q parameters to take any value between 0 and 2
# p = d = q = range(0, 2)

# # Generate all different combinations of p, q and d triplets
# pdq = list( itertools.product(p, d, q))

# # Generate all different combinations of seasonal p, q and d triplets
# seasonal_pdq = [ (x[0], x[1], x[2], 7)  for x in pdq ]

# print('Example of parameter combinations for Seasonal ARIMA ...')
# print('SARIMAX:  {} x {}'.format(pdq[1], seasonal_pdq[1]) )
# print('SARIMAX:  {} x {}'.format(pdq[1], seasonal_pdq[2]) )
# print('SARIMAX:  {} x {}'.format(pdq[2], seasonal_pdq[3]) )
# print('SARIMAX:  {} x {}'.format(pdq[2], seasonal_pdq[4]) )

In [55]:
# # AIC(통계 모델을 성능을 측정하는 기준) - 낮을수록 좋음

# select_candi = 10000000
# param_candi = ( 0, 0, 0 )
# param_seasonal_candi = ( 0, 0, 0)

# count=0
# end_count = len(pdq)

# for param in pdq:   
#     for param_seasonal in seasonal_pdq:
#         try:
#             mod = sm.tsa.statespace.SARIMAX(data['price'],
#                                             order=param,
#                                             seasonal_order=param_seasonal,
#                                             enforce_stationarity=False,
#                                             enforce_invertibility=False
#                                            )
#             results = mod.fit()
#             count += 1
#             if count <= 5:
#                 print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
            
#             if results.aic < select_candi:
#                 select_candi = results.aic
#                 param_candi = param
#                 param_seasonal_candi = param_seasonal
#         except:
#             continue
            
# print(param_candi, param_seasonal_candi, select_candi) 

In [56]:
# def predict(exog1, exog2):

#     mod = sm.tsa.statespace.SARIMAX(train, exog = exog1, order = (1, 0, 1), seasonal_order = (0, 1, 1, 7))
#     fit_res = mod.fit(disp = False)

#     forecast = fit_res.forecast(steps = 31, exog = exog2)

#     # 2021년 실제값
#     actual = data['price'][-31:]

#     # # 2022년 실제값
#     # actual = [2619529000, 2480632000, 2778454000, 2637731000, 2831080000]

#     rmse = np.sqrt(mean_squared_error(actual, forecast))
    
#     return rmse

# # order = (1,0,0), seasonal_order = (2,0,1,7)
# # order = (1, 0, 1), seasonal_order = (0, 1, 1, 7)

In [57]:
# predict(exog1, exog2)

# order = (1,0,0), seasonal_order = (2,0,1,7) / order = (1, 0, 1), seasonal_order = (0, 1, 1, 7)

# 새로운 arima 코드.ipynb / 찬재.ipynb

# 변수없이 12032081.903667446 / 12822899.653348675
# 전체평균 11134711.02414859 / 14996102.978741448
# 전체평균제외 13945093.033749819 / 15015795.613447221
# 온도만 13141867.131440545 / 13840825.71410531
# 습도만 12218263.756765371 / 13670886.534086179

In [58]:
mod = sm.tsa.statespace.SARIMAX(train, exog = exog1, order = (1, 0, 0), seasonal_order = (2, 0, 1, 7))
fit_res = mod.fit(disp = False)

forecast = fit_res.forecast(steps = 365, exog = exog2)

forecast

c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


2022-01-01    5.269526e+07
2022-01-02    4.893632e+07
2022-01-03    3.619111e+07
2022-01-04    2.715805e+07
2022-01-05    3.009494e+07
                  ...     
2022-12-27    5.909187e+05
2022-12-28    1.017951e+06
2022-12-29    9.727240e+05
2022-12-30    3.054929e+06
2022-12-31    3.089745e+06
Freq: D, Name: predicted_mean, Length: 365, dtype: float64

In [59]:
forecast.to_csv('전체평균.csv', index=False)